In [128]:
import requests
import json
import pandas as pd
import pymysql
import time
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

cityhall_conn_info = {}
cityhall_conn_info['host'] = 'end_point'
cityhall_conn_info['user'] = 'user_id'
cityhall_conn_info['password'] = 'password'
cityhall_conn_info['database'] = 'db_name'

In [126]:
def fetch_data(query, conn_info):
    # MySQL 데이터베이스에 접속
    connection = pymysql.connect(host = conn_info['host'],
                                 user = conn_info['user'],
                                 password = conn_info['password'],
                                 database = conn_info['database'],
                                 cursorclass=pymysql.cursors.DictCursor)

    try:
        # 커서 객체 생성
        with connection.cursor() as cursor:
            # SQL 쿼리 실행
            sql = query
            cursor.execute(sql)

            # 결과 가져오기
            result = cursor.fetchall()

        # 변경사항을 데이터베이스에 커밋
        connection.commit()

    finally:
        # 데이터베이스 연결 종료
        connection.close()

    df = pd.DataFrame(result)

    return df

In [170]:
# mysql DB의 department_neis_info 테이블을 불러옴.
code_df = fetch_data("select eudcation_office_code, administration_std_code, english_name from department_neis_info", cityhall_conn_info)

In [207]:
code_df.head()

,eudcation_office_code,administration_std_code,english_name
0,B10,7010059,Kyunggi High School
1,J10,7551010,Saetbyeol Middle School
2,J10,7551030,Youngsung Middle School
3,J10,7531100,GwangHwi High School
4,F10,7391154,Salesian Girls Middle School


In [199]:
# NEIS 급식 API 주소값
url_school_meal = "http://open.neis.go.kr/hub/mealServiceDietInfo"

# NEIS 학교정보 API 주소값
# url_school_info = "http://open.neis.go.kr/hub/schoolInfo"

# 호출된 결과 JSON key 이름
json_key_name = url_school_meal.split('/')[-1]

# NEIS API의 키 입력
service_key = "MY_KEY"

res_df = pd.DataFrame()

start_time = time.time()  # for문 시작 전 시간 측정

# 누비랩 DB에 존재하는 학교코드와 교육청코드를 순회하면서 for문 실행
for i in range(len(code_df)):
    eoffice_code = code_df.iloc[i]['eudcation_office_code'] # 교육청 코드
    school_code = code_df.iloc[i]['administration_std_code'] # 학교 코드
    school_name = code_df.iloc[i]['english_name'] # 학교이름(영어)
    
    pidx_lst = [1,2]
    
    # 1000개(한 번에 불러올 수 있는 데이터 최대 값) 이상의 데이터가 있을 것을 대비해 
    for pidx_value in pidx_lst:
        
        # pIndex(페이지 위치), pSize(페이지당 요청 숫자)는 교육정보 개방포털 Open API의 기본적인 필수 요청인자입니다. 
        # 호출하려는 데이터의 페이지 위치와 페이지당 요청 숫자를 뜻하는 것으로 총100건의 데이터가 있다면 
        # 페이지 사이즈가 10이고 페이지 인덱스를 8로 잡았을시, 71~80번째의 데이터를 보여줍니다. 
        # 다른 예로 페이지 사이즈가 5, 페이지 인덱스를 10으로 잡았을시, 46~50번째의 데이터를 보여줍니다.
        params_school_meal = {
            'KEY':service_key,
            'Type':'json',
            'pIndex':str(pidx_value),
            'pSize':'1000',
            'ATPT_OFCDC_SC_CODE':str(eoffice_code),
            'SD_SCHUL_CODE':str(school_code),
        }
        
        # 학교 정보 API parameter
        # params_school_info = {
        #     'KEY':service_key,
        #     'Type':'json',
        #     'pIndex':'13',
        #     'pSize':'1000'
        # }

        try:
            # get방식으로 호출
            response = requests.get(url_school_meal, params=params_school_meal)

            # 결과 반환
            contents = response.text

            # JSON 문자열을 Python 딕셔너리로 변환
            data = json.loads(contents)

            # 'row' 키에 해당하는 데이터를 추출하여 DataFrame으로 변환
            df = pd.DataFrame(data[f"{json_key_name}"][1]['row'])

            # 데이터 병합
            res_df = pd.concat([res_df, df],axis=0)

        except Exception as E:
            print(f"{school_name} API 호출 중 오류 발생: {E}")
            continue
        
        time.sleep(2)
        print(f"{school_name} {pidx_value}번째 API 호출. 데이터 개수: {len(df)}개")
                
        if len(df) < 1000: # 데이터가 1000개 미만이면 더이상 그 학교의 데이터가 없으므로 break해서 for문 탈출
            print('break')
            break
        if (pidx_value == pidx_lst[-1]) and (len(df) == 1000): # 데이터가 1000개 이고, pidx값이 pidx_lst에서 마지막값이면, pidx_lst에 다음 숫자 append
            pidx_lst.append(pidx_value+1)
        if (pidx_value != pidx_lst[-1]) and (len(df) == 1000): # 데이터가 1000개 이고, pidx값이 pidx_lst의 마지막값이 아니라면, 그대로 진행 
            pass    
        
    
end_time = time.time()  # for문 종료 후 시간 측정
elapsed_time = end_time - start_time  # 경과 시간 계산
elapsed_minutes = elapsed_time / 60 # 초를 분으로 변환
print(f"작업 총 실행 시간: {elapsed_minutes:.2f}분")

Kyunggi High School 1번째 API 호출. 데이터 개수: 643개
break
Saetbyeol Middle School 1번째 API 호출. 데이터 개수: 584개
break
Youngsung Middle School 1번째 API 호출. 데이터 개수: 582개
break
GwangHwi High School 1번째 API 호출. 데이터 개수: 578개
break
Salesian Girls Middle School 1번째 API 호출. 데이터 개수: 272개
break
songsan elementary school  1번째 API 호출. 데이터 개수: 601개
break
Seoul Nowon Elementary School 1번째 API 호출. 데이터 개수: 583개
break
Pusan Foreign Language High School 1번째 API 호출. 데이터 개수: 1000개
Pusan Foreign Language High School 2번째 API 호출. 데이터 개수: 256개
break
Goyangogeum Elementary School 1번째 API 호출. 데이터 개수: 599개
break
Wansan Girls High School 1번째 API 호출. 데이터 개수: 48개
break
GAON High School 1번째 API 호출. 데이터 개수: 1000개
GAON High School 2번째 API 호출. 데이터 개수: 934개
break
Busan International High School 1번째 API 호출. 데이터 개수: 1000개
Busan International High School 2번째 API 호출. 데이터 개수: 1000개
Busan International High School 3번째 API 호출. 데이터 개수: 44개
break
Namsung Girls High School 1번째 API 호출. 데이터 개수: 1000개
Namsung Girls High School 2번째 API 호출. 데이터 개수

In [208]:
res_df

,ATPT_OFCDC_SC_CODE,ATPT_OFCDC_SC_NM,SD_SCHUL_CODE,SCHUL_NM,MMEAL_SC_CODE,MMEAL_SC_NM,MLSV_YMD,MLSV_FGR,DDISH_NM,ORPLC_INFO,CAL_INFO,NTR_INFO,MLSV_FROM_YMD,MLSV_TO_YMD,LOAD_DTM
0,B10,서울특별시교육청,7010059,경기고등학교,2,중식,20210104,110.0,혼합잡곡밥*5.<br/>떡만둣국*1.5.6.10.16.18.<br/>배추겉절이*13...,쌀 : 국내산<br/>김치류 : 국내산<br/>고춧가루(김치류) : 국내산<br/>...,2609.1 Kcal,탄수화물(g) : 208.8<br/>단백질(g) : 34.3<br/>지방(g) : ...,20210104,20210104,20210111043017
1,B10,서울특별시교육청,7010059,경기고등학교,2,중식,20210105,90.0,흑미밥*<br/>청경채찜*5.6.13.18.<br/>셀프야채무쌈말이&땅콩참깨D*1....,쌀 : 국내산<br/>김치류 : 국내산<br/>고춧가루(김치류) : 국내산<br/>...,825.9 Kcal,탄수화물(g) : 146.1<br/>단백질(g) : 42.1<br/>지방(g) : ...,20210105,20210105,20210112043029
2,B10,서울특별시교육청,7010059,경기고등학교,2,중식,20210111,840.0,간장달걀밥*1.5.<br/>맑은미역국*5.6.<br/>매운갈비찜*5.6.10.13....,쌀 : 국내산<br/>김치류 : 국내산<br/>고춧가루(김치류) : 국내산<br/>...,1259.4 Kcal,탄수화물(g) : 114.6<br/>단백질(g) : 61.9<br/>지방(g) : ...,20210111,20210111,20210118043017
3,B10,서울특별시교육청,7010059,경기고등학교,2,중식,20210112,840.0,기장밥*<br/>아욱된장국*5.6.<br/>양배추쌈*5.6.13.<br/>잡채*1....,쌀 : 국내산<br/>김치류 : 국내산<br/>고춧가루(김치류) : 국내산<br/>...,1350.9 Kcal,탄수화물(g) : 174.0<br/>단백질(g) : 68.1<br/>지방(g) : ...,20210112,20210112,20210119043031
4,B10,서울특별시교육청,7010059,경기고등학교,2,중식,20210113,840.0,장조림버터비빔밥*1.2.5.10.<br/>미니온모밀*3.5.6.7.13.16.18....,쌀 : 국내산<br/>김치류 : 국내산<br/>고춧가루(김치류) : 국내산<br/>...,1555.3 Kcal,탄수화물(g) : 190.6<br/>단백질(g) : 83.6<br/>지방(g) : ...,20210113,20210113,20210120043016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,J10,경기도교육청,7530609,양곡고등학교,3,석식,20240423,160.0,치킨마요덮밥 (1.5.6.13.15)<br/>콩가루배추국(맑음) (5.6)<br/>...,쇠고기(종류) : 국내산(한우)<br/>쇠고기 식육가공품 : 국내산<br/>돼지고기...,1344.8 Kcal,탄수화물(g) : 148.7<br/>단백질(g) : 53.2<br/>지방(g) : ...,20240423,20240423,20240408
14,J10,경기도교육청,7530609,양곡고등학교,3,석식,20240424,160.0,기장밥 <br/>햄모듬찌개 (1.2.5.6.9.10.13.15.16)<br/>만두탕...,쇠고기(종류) : 국내산(한우)<br/>쇠고기 식육가공품 : 국내산<br/>돼지고기...,898.6 Kcal,탄수화물(g) : 99.5<br/>단백질(g) : 35.2<br/>지방(g) : 3...,20240424,20240424,20240408
15,J10,경기도교육청,7530609,양곡고등학교,3,석식,20240426,160.0,쌀밥 <br/>감자미역국 (5.6)<br/>부추장떡 (5.6.13.16)<br/>훈...,쇠고기(종류) : 국내산(한우)<br/>쇠고기 식육가공품 : 국내산<br/>돼지고기...,965.0 Kcal,탄수화물(g) : 107.9<br/>단백질(g) : 37.8<br/>지방(g) : ...,20240426,20240426,20240408
16,J10,경기도교육청,7530609,양곡고등학교,3,석식,20240429,160.0,현미밥 <br/>북어국 (1.5.9)<br/>돈육간장불고기 (5.6.10.13)<b...,쇠고기(종류) : 국내산(한우)<br/>쇠고기 식육가공품 : 국내산<br/>돼지고기...,1156.6 Kcal,탄수화물(g) : 163.2<br/>단백질(g) : 54.4<br/>지방(g) : ...,20240429,20240429,20240408
